In [ ]:
!pip install --upgrade mxnet-cu102mkl gluoncv
!pip install mxnet-cu100
!pip install Cython
!pip install mmcv

     |████████████████████████████████| 750.3MB 17kB/s 
     |████████████████████████████████| 1.0MB 49.1MB/s 
     |████████████████████████████████| 11.8MB 53.6MB/s 
     |████████████████████████████████| 245kB 56.7MB/s 
     |████████████████████████████████| 317kB 49.6MB/s 
     |████████████████████████████████| 215kB 56.9MB/s 
     |████████████████████████████████| 102kB 14.1MB/s 
     |████████████████████████████████| 665kB 44.7MB/s 
     |████████████████████████████████| 133kB 55.5MB/s 
     |████████████████████████████████| 972kB 52.8MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.6MB 49.1MB/s 
     |████████████████████████████████| 962kB 48.7MB/s 
     |████████████████████████████████| 71kB 10.8MB/s 
     |████████████████████████████████| 7.0MB 46.6MB/s 
     |████████████████████████████████| 71kB 10.4MB/s 
     |█████████████████████████████

In [1]:
from __future__ import division

import argparse, time, logging, os, sys, math

import numpy as np
import mxnet as mx
import gluoncv as gcv
from mxnet import gluon, nd, init, context
from mxnet import autograd as ag
from mxnet.gluon import nn
from mxnet.gluon.data.vision import transforms

from gluoncv.data.transforms import video
from gluoncv.data import VideoClsCustom
from gluoncv.model_zoo import get_model
from gluoncv.utils import makedirs, LRSequential, LRScheduler, split_and_load, TrainingHistory

ModuleNotFoundError: ignored

In [ ]:
root_path = '/content/drive/MyDrive/vin/DeepLearning/final_project/my_dataset/processed/'
setting_path = '/content/drive/MyDrive/vin/DeepLearning/final_project/my_dataset/train_file.txt'
output_path = '/content/drive/MyDrive/vin/DeepLearning/final_project/output/'

num_gpus = 1
ctx = [mx.gpu(i) for i in range(num_gpus)]
transform_train = video.VideoGroupTrainTransform(size=(224, 224), scale_ratios=[1.0, 0.8], mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
per_device_batch_size = 5
num_workers = 0
batch_size = per_device_batch_size * num_gpus

train_dataset = VideoClsCustom(root=os.path.expanduser(root_path),
                               setting=os.path.expanduser(setting_path),
                               train=True,
                               video_loader=True,
                               new_length=32,
                               transform=transform_train)
print('Load %d training samples.' % len(train_dataset))
train_data = gluon.data.DataLoader(train_dataset, batch_size=batch_size,
                                   shuffle=True, num_workers=num_workers)

num_gpus = 1
ctx = [mx.gpu(i) for i in range(num_gpus)]
transform_train = video.VideoGroupTrainTransform(size=(224, 224), scale_ratios=[1.0, 0.8], mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
per_device_batch_size = 5
num_workers = 0
batch_size = per_device_batch_size * num_gpus

net = get_model(name='i3d_resnet50_v1_custom', nclass=3)
net.collect_params().reset_ctx(ctx)

# Learning rate decay factor
lr_decay = 0.1
# Epochs where learning rate decays
lr_decay_epoch = [40, 80, 100]

# Stochastic gradient descent
optimizer = 'sgd'
# Set parameters
optimizer_params = {'learning_rate': 0.001, 'wd': 0.0001, 'momentum': 0.9}

# Define our trainer for net
trainer = gluon.Trainer(net.collect_params(), optimizer, optimizer_params)

loss_fn = gluon.loss.SoftmaxCrossEntropyLoss()

train_metric = mx.metric.Accuracy()
train_history = TrainingHistory(['training-acc'])

epochs = 20
lr_decay_count = 0

for epoch in range(epochs):
    tic = time.time()
    train_metric.reset()
    train_loss = 0

    # Learning rate decay
    if epoch == lr_decay_epoch[lr_decay_count]:
        trainer.set_learning_rate(trainer.learning_rate*lr_decay)
        lr_decay_count += 1

    # Loop through each batch of training data
    for i, batch in enumerate(train_data):
        # Extract data and label
        data = split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
        label = split_and_load(batch[1], ctx_list=ctx, batch_axis=0)

        # AutoGrad
        with ag.record():
            output = []
            for _, X in enumerate(data):
                X = X.reshape((-1,) + X.shape[2:])
                pred = net(X)
                output.append(pred)
            loss = [loss_fn(yhat, y) for yhat, y in zip(output, label)]

        # Backpropagation
        for l in loss:
            l.backward()

        # Optimize
        trainer.step(batch_size)

        # Update metrics
        train_loss += sum([l.mean().asscalar() for l in loss])
        train_metric.update(label, output)

        if i == 100:
            break

    name, acc = train_metric.get()

    # Update history and print metrics
    train_history.update([acc])
    print('[Epoch %d] train=%f loss=%f time: %f' %
        (epoch, acc, train_loss / (i+1), time.time()-tic))

# We can plot the metric scores with:
train_history.plot()

Load 427 training samples.


100%|██████████| 55344/55344 [00:00<00:00, 85082.81KB/s]


conv0_weight is done with shape:  (64, 3, 5, 7, 7)
batchnorm0_gamma is done with shape:  (64,)
batchnorm0_beta is done with shape:  (64,)
batchnorm0_running_mean is done with shape:  (64,)
batchnorm0_running_var is done with shape:  (64,)
layer1_0_conv0_weight is done with shape:  (64, 64, 3, 1, 1)
layer1_0_batchnorm0_gamma is done with shape:  (64,)
layer1_0_batchnorm0_beta is done with shape:  (64,)
layer1_0_batchnorm0_running_mean is done with shape:  (64,)
layer1_0_batchnorm0_running_var is done with shape:  (64,)
layer1_0_conv1_weight is done with shape:  (64, 64, 1, 3, 3)
layer1_0_batchnorm1_gamma is done with shape:  (64,)
layer1_0_batchnorm1_beta is done with shape:  (64,)
layer1_0_batchnorm1_running_mean is done with shape:  (64,)
layer1_0_batchnorm1_running_var is done with shape:  (64,)
layer1_0_conv2_weight is done with shape:  (256, 64, 1, 1, 1)
layer1_0_batchnorm2_gamma is done with shape:  (256,)
layer1_0_batchnorm2_beta is done with shape:  (256,)
layer1_0_batchnorm2_ru

100%|██████████| 208483/208483 [00:02<00:00, 81017.06KB/s]


[Epoch 0] train=0.688525 loss=0.730142 time: 2215.500100
[Epoch 1] train=0.911007 loss=0.290713 time: 624.069623
[Epoch 2] train=0.964871 loss=0.122777 time: 529.389755
[Epoch 3] train=0.983607 loss=0.065077 time: 449.736710
[Epoch 4] train=0.985948 loss=0.047646 time: 444.775362
[Epoch 5] train=0.995316 loss=0.027196 time: 426.862720


RuntimeError: ignored

In [ ]:
net.save_parameters(output_path+'i3d_resnet50_v1_custom_20epochs.params')